In [3]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [4]:
def make_data():
    
	imageData=[]
	labels=[]
	data=[]
	features=28*28*3
	noImage=96000
	nClass=96
	path=os.getcwd()
	folderpath=os.path.join(path,"classes")
	vidpath=os.path.join(path,"vimage")

	label=-1
	for length in range(2):
		for width in range(2):
			for angle in range(12):
				for color in range(2):

					string=str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
					fpath=folderpath+"/"+string

					ipath=fpath+"/"+str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
					label=label+1

					for image in range(1000):
						iName=ipath+"_"+str(image)+".jpeg"
						img_data=cv2.imread(iName)
						data.append(img_data.flatten().reshape(features))
						labels.append(label)
	imageData=np.array(data,float).reshape(noImage,features)
	labels=np.array(labels,float).reshape(noImage,1)
	imageData=imageData/255.0
	data=np.concatenate((imageData,labels),axis=1)
	np.random.shuffle(data)
	data=data.reshape(noImage,features+1)


	ipt=data[:,:-1]
	opt=np.array(data[:,-1],int).reshape(noImage,1)
	b=np.zeros((noImage,nClass),int)
	for i in range(noImage):
		b[i,opt[i]]=1

	return ipt,opt

In [49]:
x,y=make_data()

In [50]:
nx = x
ny = y

print(nx.shape)
print(ny.shape)

(96000, 2352)
(96000, 1)


In [51]:
flat_list = []
for sublist in ny:
    for item in sublist:
        flat_list.append(item)
zy = flat_list
# print(y.shape)
print(zy[np.argmax(zy)])

95


In [52]:
splitSize = int(nx.shape[0]*0.7)
nx, testData = nx[:splitSize,:], nx[splitSize:,:]
zy, testY = zy[:splitSize], zy[splitSize:]
print(zy[:20])

[78, 81, 14, 9, 10, 84, 34, 94, 45, 14, 62, 38, 77, 26, 80, 65, 6, 55, 29, 29]


In [54]:
print(nx.shape)
# print(zy.shape)
nx = nx.astype('float32')
nx /= 255
nx = nx.reshape(nx.shape[0], 28, 28, 3)
print(nx.shape)
zy = np.array(zy)
print(zy.shape)

# testData = testData.astype('float32')
# testData /= 255
# testData = testData.reshape(x.shape[0], 28, 28, 1)

(67200, 2352)
(67200, 28, 28, 3)
(67200,)


In [55]:
# print(y[:10])
# y_lab = []
# cnt = 0
# for val in y:
#     if cnt < 10:
#         print(val)
#         cnt += 1
#     y_lab.append(np.argmax(val)+1)
# y = np.array(y_lab)

# print(y[:10])

# y_lab = []
# for val in testY:
#     y_lab.append(np.argmax(val)+1)
# testY = y_lab

In [ ]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv2D(2, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
# model2.add(tf.keras.layers.Conv2D(36, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
# model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
# model2.add(tf.keras.layers.Conv2D(36, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
# model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
# model2.add(tf.keras.layers.BatchNormalization())
# model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(1024, activation="relu"))

model2.add(tf.keras.layers.Dense(96, activation='softmax'))

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()

# print(y_train.shape)

model2.fit(nx, zy, batch_size=64, epochs=1)

score = model2.evaluate(testData, testY)
# pred = model2.predict(testData)
print('\n', 'Test Acc = ', score[1])
# print('\n'," Predictions: ", pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 2)         296       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 2)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 392)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              402432    
_________________________________________________________________
dense_7 (Dense)              (None, 96)                98400     
Total params: 501,128
Trainable params: 501,128
Non-trainable params: 0
_________________________________________________________________
62976/67200 [===========================>..] - ETA: 4s - loss: 3.2995 - acc: 0.0929